# Topic 5. Recursive functions and data types

The goals of this topic are to understand:

* How recursive types (lists, trees, etc.) are defined algebraically
* How functions over recursive types are defined recursivelly
* The two major types of recursive functions: general and tail-recursive

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 16. Working with Lists
- Chapter 26. Extractors (optional)

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapters 29-36. Recursion.

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



[]            - 1
[1]           - |Int|
[1,2]         - |Int|*|Int|
[3,4,5]       - |Int|*|Int|*|Int|
...

|List[Int]| = 1 + |Int| + |Int|*|Int| + |Int|*|Int|*|Int| + ...

List[Int] = Unit + Int*Unit + Int*Int + Int*Int*Int + ....
   = Unit + Int*(Unit + Int + Int*Int + ...)
   = Unit + Int*List[Int]

The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [ ]:
object StdDefinition:
    enum IntList: 
        case Empty()
        case NonEmpty(head: Int, tail: IntList)

In [ ]:
import StdDefinition.IntList._

// [1,2,3]
NonEmpty(1, NonEmpty(2, NonEmpty(3, Empty())))

In [ ]:
object StdDefinition:
    enum List[A]: 
        case Empty()
        case NonEmpty(head: A, tail: List[A])

In [ ]:
object StdDefinition:
    enum List[A]: 
        case Nil()
        case ::(head: A, tail: List[A])

In [ ]:
val l: List[Int] = Nil
val l1: List[Int] = ::(1, ::(2, ::(3, Nil)))

In [ ]:
val l: List[Int] = Nil
val l1: List[Int] = 1 :: (2 :: (3 :: Nil))
val l2: List[Int] = List(1, 2, 3)

In [ ]:
def isLeft(e: Either[Int, String]): Boolean = 
    e match 
        case Left(_) => true
        case Right(_) => true

In [ ]:
def isEmpty(l: List[Int]): Boolean = 
    l match 
        case Nil => true 
        case ::(head, tail) => false

In [ ]:
def isEmpty(l: List[Int]): Boolean = 
    l match 
        case Nil => true 
        case head :: tail => false

In [ ]:
List(1,2) match 
    case List(_, _, _) => "yes"
    case _ => "no"

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [ ]:
object ActualStdDefinition:


### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [ ]:
// Less beautifully 

// More idiomatically


And we can also pattern match on lists, similarly:

In [ ]:
// Less beautifully


// more idiomatically


// or



##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [2]:
// Using mutable variables

def lengthI[A](l: List[A]): Int = 
    var out: Int = 0
    var aux: List[A] = l
    while aux != Nil do 
        out += 1
        aux = aux.tail

    /*return*/ out


defined function lengthI

In [ ]:
// invoke
lengthI(List(1,2,3,4,5))
lengthI(List())
lengthI(List('1', 'd', 'w'))

The recursive function is implemented as follows: 

In [ ]:
// Using recursive functions
def lengthR[A](l: List[A]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => 
            val tailSol: Int = lengthR(tail)
            ??? : Int


In [3]:
// Using recursive functions
def lengthR[A](l: List[A]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailSol: Int = lengthR(tail)
            tailSol + 1 : Int

defined function lengthR

In [ ]:
// Using recursive functions
//@annotation.tailrec
def lengthR[A](l: List[A]): Int = 
    l match 
        case Nil => 0
        case head :: tail => 
            lengthR(tail) + 1

In [ ]:
lengthR(List(1,2,3,4)) == lengthI(List(1,2,3,4))

In [ ]:
lengthI(List.fill(100000)(0))

In [ ]:
lengthR(List.fill(10000)(0))

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that are available, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with very long lists.

### Tail-recursive functions

The implementation using tail-recursion solves the issues with the stack. It commonly makes use of auxiliary functions:

In [4]:
// Using tail-recursive functions
def lengthTR[A](l: List[A]): Int = 

    @annotation.tailrec
    def step(out: Int, aux: List[A]): Int = 
        if aux == Nil then out
        else step(out + 1, aux.tail)

    step(0, l)

/*
    var out: Int = 0
    var aux: List[A] = l
    while aux != Nil do 
        out += 1
        aux = aux.tail
    out
*/

defined function lengthTR

In [ ]:
// Using tail-recursive functions
def lengthTR[A](l: List[A]): Int = 

    @annotation.tailrec
    def step(out: Int, aux: List[A]): Int = 
        if aux == Nil then out
        else step(???(out, aux.head), aux.tail)

    step(???, l)

/*
    var out: Int = 0
    var aux: List[A] = l
    while aux != Nil do 
        out += 1
        aux = aux.tail
    out
*/

In [ ]:
// Using tail-recursive functions
def lengthTR[A](l: List[A]): Int = 

    @annotation.tailrec
    def step(out: Int, aux: List[A]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(out, head), tail)

    step(???, l)

-- [E050] Type Error: cell1.sc:8:21 --------------------------------------------
8 |                step(???(out, head), tail)
  |                     ^^^
  |                     method ??? in object Predef does not take parameters
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [ ]:
lengthTR(List.fill(1000000)(1))

We can check the stack-safety problems of non-tail recursive functions by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [ ]:
// First, imperatively



In [ ]:
// Next, tail-recursively



We can also use the function [`fill`](https://www.scala-lang.org/api/2.13.3/scala/collection/immutable/List$.html#fill[A](n:Int)(elem:=%3EA):CC[A]) of the Scala standard library.

Now, let's calculate the length of a list long enough to blow up the stack, using each of the three implementations:

In [ ]:
// Imperatively


In [ ]:
// Tail-recursive


In [ ]:
// Plain recursive


### Unit testing with `scalatest`

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


In [16]:
class TestLength(len: [A] => List[A] => Int) extends AnyFlatSpec with should.Matchers: 
    "Imperative version length" should "work" in:
        len[Char](List()) shouldBe 0
        len[Int](List(1,2,3)) shouldBe 3

defined class TestLength

In [15]:
class TestLength(len: List[Int] => Int) extends AnyFlatSpec with should.Matchers: 
    "Imperative version length" should "work" in:
        len(List()) shouldBe 0
        len(List(1,2,3)) shouldBe 3

defined class TestLengthI

In [16]:
class TestLength[A](len: List[A] => Int) extends AnyFlatSpec with should.Matchers: 
    "Imperative version length" should "work" in:
        len(List()) shouldBe 0
        len(List('1','2','3')) shouldBe 3

-- [E007] Type Mismatch Error: cell17.sc:4:17 ----------------------------------
4 |        len(List('1','2','3')) shouldBe 3
  |                 ^^^
  |      Found:    ('1' : Char)
  |      Required: A
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
-- [E007] Type Mismatch Error: cell17.sc:4:21 ----------------------------------
4 |        len(List('1','2','3')) shouldBe 3
  |                     ^^^
  |      Found:    ('2' : Char)
  |      Required: A
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
-- [E007] Type Mismatch Error: cell17.sc:4:25 ----------------------------------
4 |        len(List('1','2','3')) shouldBe 3
  |                         ^^^
  |      Found:    ('3' : Char

In [10]:
object TestLengthI extends AnyFlatSpec with should.Matchers: 
    "Imperative version length" should "work" in:
        lengthI(List()) shouldBe 0
        lengthI(List(1,2,3)) shouldBe 3

defined object TestLengthI

In [11]:
object TestLengthR extends AnyFlatSpec with should.Matchers: 
    "Imperative version length" should "work" in:
        lengthR(List()) shouldBe 0
        lengthR(List(1,2,3)) shouldBe 3

defined object TestLengthR

In [13]:
object TestLengthTR extends AnyFlatSpec with should.Matchers: 
    "Imperative version length" should "work" in:
        lengthTR(List()) shouldBe 0
        lengthTR(List(1,2,3)) shouldBe 3

defined object TestLengthTR

In [14]:
run(TestLengthTR)

cell13$Helper$TestLengthTR:
Imperative version length
- should work


From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. The test catalogue receives the actual function to be tested as a parameter. For instance, this is a possible test class for the `length` function:

In [ ]:
class TestLength(lengthF: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "length" should "work" in:
        ??? 

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [ ]:
run(TestLength(lengthR))

### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [17]:
class TestSum(sum: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "sum" should "work" in:
        sum(List()) shouldBe 0
        sum(List(1,2,3,4)) shouldBe 10
        sum(List(1)) shouldBe 1

defined class TestSum

In [19]:
// Recursively

def sum(l: List[Int]): Int = 
    l match 
        case Nil => ??? : Int
        case head :: tail => 
            val tailSol: Int = sum(tail)
            ??? : Int

defined function sum

In [22]:
// Recursively

def sumR(l: List[Int]): Int = 
    l match 
        case Nil => 0 : Int
        case head :: tail => 
            val tailSol: Int = sum(tail)
            head + tailSol : Int

defined function sumR

In [31]:
// Recursively

def sumR(l: List[Int]): Int = 
    l match 
        case Nil => 0
        case head :: tail => 
            head + sum(tail)

defined function sumR

In [21]:
run(TestSum(sumR))

cell17$Helper$TestSum:
sum
- should work


In [ ]:
// With tail-recursion

def sumTR(l: List[Int]): Int = 
    
    def step(out: Int, aux: List[Int]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(out, head), tail)
        
    step(???, l)


In [25]:
// With tail-recursion

def sumTR(l: List[Int]): Int = 
    
    @annotation.tailrec
    def step(out: Int, aux: List[Int]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(out+head, tail)
        
    step(0, l)


defined function sumTR

In [26]:
run(TestSum(sumTR))

cell17$Helper$TestSum:
sum
- should work


### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [27]:
class TestProduct(product: List[Int] => Int) extends AnyFlatSpec with should.Matchers:
    "product" should "work" in:
        product(List(1,2,3)) shouldBe 6
        product(List(4,1,2,3)) shouldBe 24
        product(List(4,1,0,2,3)) shouldBe 0
        product(List()) shouldBe 1

defined class TestProduct

 This is the common recursive implementation:

In [28]:
def product(l: List[Int]): Int = 
    l match 
        case Nil => ???
        case head :: tail => 
            val tailSol: Int = product(tail)
            ???

defined function product

In [29]:
def product(l: List[Int]): Int = 
    l match 
        case Nil => 1
        case head :: tail => 
            val tailSol: Int = product(tail)
            head * tailSol

defined function product

In [32]:
def product(l: List[Int]): Int = 
    l match 
        case Nil => 1
        case head :: tail => 
            if head == 0 then 0
            else 
                val tailSol: Int = product(tail)
                head * tailSol

defined function product

In [32]:
def product(l: List[Int]): Int = 
    l match 
        case Nil => 1
        case head :: tail if head == 0 => 0
        case head :: tail => 
            val tailSol: Int = product(tail)
            head * tailSol

defined function product

In [34]:
def product(l: List[Int]): Int = 
    l match 
        case Nil => 1
        case 0 :: tail => 0
        case head :: tail => 
            val tailSol: Int = product(tail)
            head * tailSol

defined function product

In [35]:
def product(l: List[Int]): Int = 
    l match 
        case Nil => 1
        case 0 :: tail => 0
        case head :: tail => 
            head * product(tail)

defined function product

In [33]:
run(TestProduct(product))

cell27$Helper$TestProduct:
product
- should work


But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [ ]:
// optimization for 0



In [ ]:
run(TestProduct(product2))

A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [ ]:
class TestMember(member: (List[Int], Int) => Boolean) extends AnyFlatSpec with should.Matchers:
    "member" should "work" in:
        ???

In [ ]:
run(TestMember(member))

We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [36]:
object Std: 
    enum Option[+A]:  // Option[A] = 1 + A
        case None 
        case Some(a: A)

defined object Std

In [37]:
class TestLast(last: List[Int] => Option[Int]) extends AnyFlatSpec with should.Matchers:
    "last" should "work" in:
        last(List(1,2,3)) shouldBe Some(3)
        last(List(1)) shouldBe Some(1)
        last(List()) shouldBe None

defined class TestLast

In [38]:
def lastR[A](l: List[A]): Option[A] = 
    l match 
        case Nil => ??? : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = lastR(tail)
            ??? : Option[A]

defined function lastR

In [43]:
def lastR[A](l: List[A]): Option[A] = 
    l match 
        case Nil => None : Option[A]
        case head :: tail => 
            val tailSol: Option[A] = lastR(tail)
            if tail == Nil then Some(head)
            else tailSol : Option[A]

defined function lastR

In [48]:
@annotation.tailrec
def lastR[A](l: List[A]): Option[A] = 
    l match 
        case Nil => None : Option[A]
        case head :: Nil => Some(head)
        case head :: tail => lastR(tail)

defined function lastR

In [49]:
run(TestLast(lastR))

cell37$Helper$TestLast:
last
- should work


In [ ]:
@annotation.tailrec


In [ ]:
run(TestLast(last))

### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [ ]:
class TestInsertLast(insertLast: (List[Int], Int) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "insertLast" should "work" in:
        ???

In [ ]:
run(TestInsertLast(insertLast))

### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [5]:
List(2,3).appended(1)
List(3,6,2) :+ 1

res5_0: List[Int] = List(2, 3, 1)
res5_1: List[Int] = List(3, 6, 2, 1)

In [6]:
class TestReverse(reverse: List[Int] => List[Int]) extends AnyFlatSpec with should.Matchers:
    "reverse" should "work" in:
        reverse(List(5,2,7,3)) shouldBe List(3,7,2,5) // == List(3,7,2) :+ 5
        reverse(List(2,3)) shouldBe List(3,2)
        reverse(1::List(2,3)) shouldBe List(3,2,1) // == List(3,2) :+ 1
        reverse(List()) shouldBe List()
        reverse(List(1)) shouldBe List(1)

defined class TestReverse

In [3]:
// Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => ??? : List[A]
        case head :: tail => 
            val tailSol: List[A] = reverse(tail)
            ??? : List[A]

defined function reverse

In [12]:
// Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => List(): List[A]
        case head :: tail => 
            val tailSol: List[A] = reverse(tail)
            // tailSol ++ List(head) : List[A]
            // head +: tailSol : List[A] == tailSol.+:(head)
            tailSol :+ head // == tailSol.:+(head)

defined function reverse

In [18]:
// Really inefficient 

def reverse[A](l: List[A]): List[A] = 
    l match 
        case Nil => List(): List[A]
        case head :: tail => 
            reverse(tail) :+ head // == tailSol.:+(head)

defined function reverse

In [13]:
run(TestReverse(reverse))

cell6$Helper$TestReverse:
reverse
- should work


In [17]:
reverse(List.fill(10000)(0))

java.lang.StackOverflowError: null

In [18]:
// Tail-recursive, efficiently

def reverse[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(???(out, head), tail)

    step(???, l)

-- [E050] Type Error: cell19.sc:6:21 -------------------------------------------
6 |                step(???(out, head), tail)
  |                     ^^^
  |                     method ??? in object Predef does not take parameters
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [22]:
// Tail-recursive, efficiently

def reverseTR[A](l: List[A]): List[A] = 
    def step(out: List[A], aux: List[A]): List[A] = 
        aux match 
            case Nil => out
            case head :: tail => 
                step(head +: out, tail)

    step(Nil, l)

defined function reverseTR

// 

reverse(List(1,2,3)) == List(3,2,1)

head out
.... ...
     Nil
1    1 :: Nil
2    2 :: 1 :: Nil
3    3 :: 2 :: 1 :: Nil 

step(Nil, 1 :: List(2,3))
step(1 :: Nil, 2 :: List(3))
step(2 :: 1 :: Nil, 3 :: Nil)
step(3 :: 2 :: 1 :: Nil, Nil)



In [23]:
run(TestReverse(reverseTR))

cell6$Helper$TestReverse:
reverse
- should work


### Example: concatenate lists

In [2]:
class TestConcatenate(concatenate: (List[Int], List[Int]) => List[Int]) 
extends AnyFlatSpec with should.Matchers:
    "concatenate" should "work" in:
        concatenate(List(2,3), List(4,5,6,7)) shouldBe List(2,3,4,5,6,7)
        concatenate(1::List(2,3), List(4,5,6,7)) shouldBe 1::List(2,3,4,5,6,7)
        concatenate(List(), List(1,2,3)) shouldBe List(1,2,3)
        concatenate(List(1,2,3), List()) shouldBe List(1,2,3)
        concatenate(List(), List()) shouldBe List()

defined class TestConcatenate

In [4]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    l1 match 
        case Nil => ??? : List[A]
        case head :: tail => 
            val tailSol: List[A] = concatenate(tail, l2)
            ??? : List[A]

defined function concatenate

In [5]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    l1 match 
        case Nil => l2 : List[A]
        case head :: tail => 
            val tailSol: List[A] = concatenate(tail, l2)
            head :: tailSol : List[A]

defined function concatenate

In [ ]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    dyv(l1)(??? : List[A]):
        (head: A, tailSol: List[A]) => 
            ???

    

In [7]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    l1.foldRight(l2: List[A]):
        (head: A, tailSol: List[A]) => 
            head :: tailSol    

defined function concatenate

In [ ]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    l1.foldRight(l2):
        (head, tailSol) => 
            head :: tailSol    

defined function concatenate

In [5]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    l1 match 
        case Nil => l2 : List[A]
        case head :: tail => 
            val tailSol: List[A] = concatenate(tail, l2)
            head :: tailSol : List[A]

defined function concatenate

In [11]:
def concatenate[A](l1: List[A], l2: List[A]): List[A] = 
    l1.foldRight(l2)(_ :: _)

defined function concatenate

In [8]:
run(TestConcatenate(concatenate))

cell2$Helper$TestConcatenate:
concatenate
- should work


Tail-recursive concatenation:

In [ ]:
run(TestConcatenate(concatenateTR))